In [ ]:
#!pip install --upgrade pika

In [ ]:
from billiard import current_process
import time
import math
import socket
import numpy as np
import os
import random
import json
from utils import NumpyEncoder

In [ ]:
import pika
pika.__version__

In [4]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [5]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [6]:
!sudo rabbitmqctl list_bindings

Listing bindings ...


In [7]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_pings	direct


In [8]:
!sudo rabbitmqctl list_bindings

Listing bindings ...


In [9]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'

parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
exchangeName = 'direct_pings'

In [10]:
def create_routing_name():
    
    #process_name=current_process()._name
    #print('process name: ', process_name)
    
    #try:
    #    process_index=str(current_process().index)
    #except:
    #    process_index='_none_'
    
    #print('process index: ', process_index)
    
    tmpip= socket.gethostbyname(socket.gethostname())
    print('ip: ', tmpip)
    print(' os_pid: ',str(os.getpid()))
    
    results='routing_'+tmpip+'_'+str(os.getpid())
    
    return results

In [11]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)    

In [12]:
def on_request(ch, method, props, body):
    global exchangeName
    global queue_name
    
    msg = str(body.decode('ascii')) # body is the message from server to the worker. 
    task=json.loads(msg)['task']
    print(" I just received message:", msg)
    
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    if task=='Ping!':
        result = create_routing_name() # Results, Fib(n), calculated
        ch.queue_bind(exchange=exchangeName, queue=queue_name,routing_key=result)
        json_dump = json.dumps({'task': 'Ping!','result':result}, cls=NumpyEncoder)
    elif task =='Fib':
        n=int(json.loads(msg)['num'])
        result = str(fib(n))
        json_dump = json.dumps({'task': 'Fib','result':result}, cls=NumpyEncoder) 
    else:
        raise Exception('Invalid task!')
        
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key=props.reply_to, # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=json_dump)
    #ch.basic_ack(delivery_tag=method.delivery_tag)
    
    print(" Calculations done. Results sent back to the server")
    print(" *** ")


In [13]:
channel.exchange_declare(exchange=exchangeName, exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange=exchangeName, queue=queue_name,routing_key='ping_queue')

#channel.basic_qos(prefetch_count=1)


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

In [14]:
#When the worker receives a message it will tun the "on_request" call back function
channel.basic_consume(queue=queue_name, on_message_callback=on_request, auto_ack=True)  

'ctag1.d2e5ac621b794f58893295ac57bb73bb'

In [ ]:
print(" [x] Awaiting requests")
channel.start_consuming()

 [x] Awaiting requests
 I just received message: {"task": "Ping!"}
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7f0c0387e978> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.d2e5ac621b794f58893295ac57bb73bb', 'delivery_tag=1', 'exchange=direct_pings', 'redelivered=False', 'routing_key=ping_queue'])>
props: <BasicProperties(['content_type=text/plain', 'correlation_id=9d6accb0-057c-49ea-a0e9-90c0ec2ac554', 'delivery_mode=1', 'reply_to=amq.gen-5hNzVPzjk0HHuj9TCfVvHw'])>
props.reply_to: amq.gen-5hNzVPzjk0HHuj9TCfVvHw
props.correlation_id: 9d6accb0-057c-49ea-a0e9-90c0ec2ac554
ip:  172.31.14.37
 os_pid:  11770
 Calculations done. Results sent back to the server
 *** 
 I just received message: {"task": "Ping!"}
ch: <BlockingChannel impl=<Channel number=1 O